In [ ]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

import sys

Spark Configuration

In [ ]:
spark_session = SparkSession.builder \
    .appName("HDFS_Connection_Test") \
    .master("yarn") \
    .config("spark.executor.instances", "2") \
    .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

File Path

In [ ]:
path = 'hdfs://<name-node>:<port>/<path>'

In [ ]:
file_contents = spark_context.wholeTextFiles(path).map(lambda x: x[1].replace('\n', '').replace('{', '').replace('}', '').replace(' ', '').split(', '))
#file_contents = spark_context.wholeTextFiles(','.join(paths)).map(lambda x: x[1].replace('\n', '').replace('{', '').replace('}', '').replace(' ', '').split(', '))

In [38]:
split_file_contents = file_contents.map(lambda x: x[0].split(','))

In [39]:
selected_elements = split_file_contents.map(lambda x: [float(x[i]) for i in [3, 4, 26, 23, 24, 25, 27, 28, 29]])

In [40]:
#selected_elements.first()

## Data Frame

In [41]:
schema = StructType([
    StructField("duration", FloatType(), True),
    StructField("end_of_fade_in", FloatType(), True),
    StructField("start_of_fade_out", FloatType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", FloatType(), True),
    StructField("mode_confidence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("time_signature", FloatType(), True),
    StructField("time_signature_confidence", FloatType(), True)
])

In [42]:
df = spark_session.createDataFrame(selected_elements, schema)

In [ ]:
num_rows = df.count()
print("Number of rows in the DataFrame:", num_rows)

In [ ]:
df.show()

# Calculation (Average)

In [ ]:
average_values = df.agg({'duration': 'avg', 'end_of_fade_in': 'avg', 'start_of_fade_out': 'avg', 
                         'loudness': 'avg', 'mode': 'avg', 'mode_confidence': 'avg', 
                         'tempo': 'avg', 'time_signature': 'avg', 
                         'time_signature_confidence': 'avg'}).collect()[0]

In [46]:
avg_df = pd.DataFrame([average_values.asDict()])

In [47]:
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 10000


In [48]:
avg_df

,avg(duration),avg(tempo),avg(end_of_fade_in),avg(start_of_fade_out),avg(time_signature),avg(time_signature_confidence),avg(mode_confidence),avg(mode),avg(loudness)
0,238.507519,122.915449,0.758616,229.975465,3.5648,0.509937,0.477784,0.6911,-10.485669


# STOP SPARK

In [ ]:
# release the cores
spark_context.stop()